In [1]:
import torch
import numpy as np

import pymongo

from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

mongodb_connectionstring="mongodb connection string"
mongodb_dbname="test"
mongodb_collname="vector_comments1"
mongodb_indexname="vector_comments1"

client = pymongo.MongoClient(mongodb_connectionstring)
collection = client[mongodb_dbname][mongodb_collname]

In [2]:
path = '/Users/dongq/Downloads/vector/naver_shopping.txt'

with open(path) as f:
   for l in f:
       line = l.strip().split("\t")

## 출처 
## https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt

result_array = []
count = 0
with open(path) as items:
    for l in items:
       result_doc = {}
       doc = l.strip().split("\t")
       doc_vector = model.encode(doc[1]).tolist()
       result_doc['rating'] = int(doc[0])
       result_doc['comment'] = doc[1]
       result_doc['embedding'] = doc_vector
       count = count + 1
       result_array.append(result_doc)

       if count % 10 == 0:
          res = collection.insert_many(result_array)
          result_array = []
          print(count)

       if count == 10:
          break;

if result_array != []:
   res = collection.insert_many(result_array)

10


## vector search index build
mongodb_indexname="vector_comments"

```bash
{
  "mappings": {
    "dynamic": true,
    "fields": {
      "embedding": {
        "type": "knnVector",
        "dimensions": 768,
        "similarity": "cosine"
      }
    }
  }
}
```

In [5]:
mongodb_dbname="test"
mongodb_collname="vector_comments"
mongodb_indexname="vector_comments"

collection = client[mongodb_dbname][mongodb_collname]

# sample searches
query = "상담사 남자분이 너무 친절해요"
query = "와이프가 너무 좋아합니다."
query = "음식관련"
query = "best"
query = "worst"

vector_query = model.encode(query).tolist()
#print(vector_query)
pipeline = [
    {
        "$search": {
            "index": mongodb_indexname,
            "knnBeta": {
                "vector": vector_query,
                "path": "embedding",
                "k": 10
            }
        }
    },
    {
        "$limit": 10
    },
    {
        "$project": {
            "embedding": 0,
            "_id": 0,
            'search_score': {
                '$meta': 'searchScore'
            }
        }
    }
]

print("*************Vector Search Query*****************")
print(query)
print("**************************************************")

print("\n")
print("*************Vector Search Result*****************")

results = collection.aggregate(pipeline)
for result in results:
    print('rating: ' + str(result['rating']))
    print('comment: ' + result['comment'])
    print('search_score: ' + str(result['search_score']))
    print("\n")

print("**************************************************")

*************Vector Search Query*****************
와이프가 너무 좋아합니다.
**************************************************


*************Vector Search Result*****************
rating: 5
comment: 집사람이 좋아라합니다
search_score: 0.8939113616943359


rating: 5
comment: 와이프가 아주 만족하면서 잘 쓰고 있습니다
search_score: 0.8710165023803711


rating: 4
comment: 프로포즈 선물인데 와이프가 넘좋아라해서 저도 기분이 넘좋네요ㅋ
search_score: 0.8625155091285706


rating: 5
comment: 와이프가 너무 만족스러워 하네요~ 귀걸이로 활용하기에도 이뻐보이네요~ 남편 또는 남친분에게 사다라고 하세요 다들~
search_score: 0.8499746322631836


rating: 5
comment: 와이프가 편해해요 가볍고 좋습니다
search_score: 0.8401122093200684


rating: 5
comment: 남편이 너무 좋아하네요
search_score: 0.8345746994018555


rating: 5
comment: 여친이 너무 좋아합니다 이뻐요~~♡
search_score: 0.8339909315109253


rating: 4
comment: 저의 집사람이 만족합니다.
search_score: 0.8334900140762329


rating: 5
comment: 생각보다 가늘지만 예쁘고 착감기네요 와이프가 좋아합니다
search_score: 0.820135235786438


rating: 5
comment: 여자친구가 진짜 좋아합니다..
search_score: 0.8110088109970093


******************************************